In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class DefaultConv2dGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, weight, bias, stride=1, padding=0):
        ctx.save_for_backward(x, weight, bias)
        ctx.stride = stride
        ctx.padding = padding
        return F.conv2d(x, weight, bias, stride, padding)

    @staticmethod
    def backward(ctx, grad_output):
        x, weight, bias = ctx.saved_tensors
        stride = ctx.stride
        padding = ctx.padding
        grad_input = grad_weight = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input = F.grad.conv2d_input(x.shape, weight, grad_output, stride=stride, padding=padding)
        if ctx.needs_input_grad[1]:
            grad_weight = F.grad.conv2d_weight(x, weight.shape, grad_output, stride=stride, padding=padding)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(dim=[0, 2, 3])
        return grad_input, grad_weight, grad_bias, None, None

class DefaultConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        super(DefaultConv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride
        self.padding = padding
        self.weight = nn.Parameter(torch.Tensor(out_channels, in_channels, *self.kernel_size))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        return DefaultConv2dGrad.apply(x, self.weight, self.bias, self.stride, self.padding)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random
import numpy as np

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR-10は3チャンネルのカラー画像
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class AdvancedConvNet(nn.Module):
    def __init__(self, conv_module):
        super(AdvancedConvNet, self).__init__()
        self.conv1 = conv_module(3, 64, kernel_size=3, padding=1)  # Input: 32x32x3, Output: 32x32x64
        self.conv2 = conv_module(64, 128, kernel_size=3, padding=1)  # Input: 32x32x64, Output: 32x32x128
        self.pool = nn.MaxPool2d(2, 2)  # Pooling reduces size by half, Output: 16x16x128
        self.conv3 = conv_module(128, 256, kernel_size=3, padding=1)  # Input: 16x16x128, Output: 16x16x256
        self.conv4 = conv_module(256, 256, kernel_size=3, padding=1)  # Input: 16x16x256, Output: 16x16x256
        # nn.MaxPool2d(2, 2)  # Pooling reduces size by half, Output: 8x8x256
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))  # After pool: 16x16x128
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))  # After pool: 8x8x256
        x = x.view(-1, 256 * 8 * 8)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(self.dropout(x)))
        x = F.relu(self.fc2(self.dropout(x)))
        x = self.fc3(x)
        return x

def train_and_evaluate(conv_module):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AdvancedConvNet(conv_module).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        accuracy = correct / len(test_loader.dataset)
        print(f'Epoch {epoch + 1}: Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')


set_seed()
print("Training with DefaultConv2d:")
train_and_evaluate(DefaultConv2d)

set_seed()
print("Training with nn.Conv2d:")
train_and_evaluate(nn.Conv2d)


Files already downloaded and verified
Files already downloaded and verified
Training with DefaultConv2d:
Epoch 1: Test Loss: 0.0214, Accuracy: 0.4981
Epoch 2: Test Loss: 0.0176, Accuracy: 0.6007
Epoch 3: Test Loss: 0.0157, Accuracy: 0.6439
Epoch 4: Test Loss: 0.0141, Accuracy: 0.6840
Epoch 5: Test Loss: 0.0128, Accuracy: 0.7078
Training with nn.Conv2d:
Epoch 1: Test Loss: 0.0214, Accuracy: 0.4981
Epoch 2: Test Loss: 0.0176, Accuracy: 0.6007
Epoch 3: Test Loss: 0.0157, Accuracy: 0.6439
Epoch 4: Test Loss: 0.0141, Accuracy: 0.6840
Epoch 5: Test Loss: 0.0128, Accuracy: 0.7078
